## Read/Load and Import 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_amz_ppd = pd.read_csv('./amz_uk_price_prediction_dataset.csv')

In [ ]:
df_amz_ppd.head()

In [ ]:
df_amz_ppd["isBestSeller"].astype(bool)

In [ ]:
df_amz_ppd.select_dtypes("number").columns

In [ ]:
df_amz_ppd.select_dtypes("number").nunique().sort_values(ascending=False)

In [ ]:
df_numerical = df_amz_ppd.select_dtypes("number")
display(df_numerical.head())

## Part 1: Analyzing Best-Seller Trends Across Product Categories

Objective: Understand the relationship between product categories and their best-seller status.

Crosstab Analysis:

Create a crosstab between the product category and the isBestSeller status.

Are there categories where being a best-seller is more prevalent?

Hint: one option is to calculate the proportion of best-sellers for each category and then sort the categories based on this proportion in descending order.

Statistical Tests:

Conduct a Chi-square test to determine if the best-seller distribution is independent of the product category.
Compute Cramér's V to understand the strength of association between best-seller status and category.
Visualizations:

Visualize the relationship between product categories and the best-seller status using a stacked bar chart.

In [ ]:
crosstab = pd.crosstab(df_amz_ppd["category"], df_amz_ppd["isBestSeller"])

#sorted by Bestseller = True
crosstab_sort = crosstab.sort_values(by=True, ascending=False)
crosstab_sort.head(20)

In [ ]:
crosstab_sort["rel_freq"] = crosstab_sort[True] / (crosstab_sort[False] + 1e-9) * 100

#sorted by relative true/false bestseller
crosstab_sort_by_rel_freq = crosstab_sort.sort_values(by="rel_freq", ascending=False)
display(crosstab_sort_by_rel_freq.head(20))

Yes, in terms of total count the top 5 is quite different as in terms of 

TOP / onl total count of true
- Grocery	                             9008	    556
- Health & Personal Care	             9017	    552
- Baby	                                 14709	    341
- Sports & Outdoors	                     835935	    330
- Power & Hand Tools	                 8353	    306


TOP5 /in realtion to false in there own category
- Grocery	                            9008 /  556 /   6.172291
- Smart Home Security & Lighting	    98   /    6 /   6.122449
- Health & Personal Care	            9017 /  552 /   6.121770
- Mobile Phone Accessories  	        248  /   11 /   4.435484
- Power & Hand Tools	                8353 /  306 /	3.663354

In [ ]:
from scipy.stats import chi2_contingency

chi2_contingency(crosstab)[1]

In [ ]:
from scipy.stats.contingency import association

association(crosstab, method = "cramer")

In [ ]:
crosstab.head(20).plot(kind="bar", stacked=True)

plt.show()

## Part 2: Exploring Product Prices and Ratings Across Categories and Brands

Objective: Investigate how different product categories influence product prices.

Preliminary Step: Remove outliers in product prices.

For this purpose, we can use the IQR (Interquartile Range) method. Products priced below the first quartile minus 1.5 times the IQR or above the third quartile plus 1.5 times the IQR will be considered outliers and removed from the dataset. The next steps will be done with the dataframe without outliers.

Hint: you can check the last Check For Understanding at the end of the lesson EDA Bivariate Analysis for a hint on how to do this.

Violin Plots:

Use a violin plot to visualize the distribution of price across different product categories. Filter out the top 20 categories based on count for better visualization.
Which product category tends to have the highest median price? Don't filter here by top categories.

Bar Charts:

Create a bar chart comparing the average price of products for the top 10 product categories (based on count).
Which product category commands the highest average price? Don't filter here by top categories.

Box Plots:

Visualize the distribution of product ratings based on their category using side-by-side box plots. Filter out the top 10 categories based on count for better visualization.
Which category tends to receive the highest median rating from customers? Don't filter here by top categories.


In [ ]:
df_amz_ppd

In [ ]:
def tukeys_test_outliers_and_clean(df_amz_ppd, price):
    Q1 = df_amz_ppd['price'].quantile(0.25)
    Q3 = df_amz_ppd['price'].quantile(0.75)
    IQR = Q3 - Q1

    # define bounds for the outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # identify outliers
    outliers = df_amz_ppd[(df_amz_ppd['price'] < lower_bound) | (df_amz_ppd['price'] > upper_bound)]

    # create a cleaned DataFrame without outliers
    cleaned_df = df_amz_ppd[(df_amz_ppd['price'] >= lower_bound) & (df_amz_ppd['price'] <= upper_bound)]

    return outliers, cleaned_df

#function using
outliers, cleaned_df = tukeys_test_outliers_and_clean(df_amz_ppd, 'price')

# show outlieres
print("Identifizierte Ausreißer:")
print(outliers)

# new, cleanded df without outlieres for violin/boxplot
print("DataFrame ohne Ausreißer:")
print(cleaned_df)


In [ ]:
# top20 cat by value count // filter
top20_categories = cleaned_df['category'].value_counts().head(20).index
top20_categories

In [ ]:
# df with all products in the cat of top20
filtered20_df = cleaned_df[cleaned_df['category'].isin(top20_categories)]
filtered20_df

In [ ]:
# df only with category & price
result20_df = filtered20_df[['category', 'price']]
result20_df

In [ ]:
#violinplot for top20 categories
sns.violinplot(data=result20_df, x="category", y="price", hue="category", palette="coolwarm", legend=False)
plt.title("top20 categories by price", fontsize=14)
plt.xticks(rotation=90)
plt.show()

- Out of the top20 categories by count, the categroy "men" has the highest median, followed by fragrances and sports/outdoor

In [ ]:
top10_categories = cleaned_df['category'].value_counts().head(10).index
top10_categories
# df with all products in the cat of top20
filtered10_df = cleaned_df[cleaned_df['category'].isin(top10_categories)]
filtered10_df

In [ ]:
result10_df = filtered10_df[['category', 'price']]
result10_df

In [ ]:
#barplot for top10 categories

sns.barplot(data = result10_df, x="category", y="price", palette = "Set3", estimator= "mean")
plt.title("top10 categories by mean price", fontsize=14)
plt.xticks(rotation=90)
plt.show()

- Out of the top10 categories by count, the categroy "sports/outdoor" has the highest average price, followed by birtday gifts and handmade gifts

In [ ]:
result10_star_df = filtered10_df[['category', 'stars']]
result10_star_df

In [ ]:
#boxplot for top10 by review/stars
sns.boxplot(data = result10_star_df, x="category", y="stars", hue="category", palette = "coolwarm", legend=False)
plt.title("top10 categories by stars per review", fontsize=14)
plt.xticks(rotation=90)
plt.show()

- Out of the top10 categories by count, 3 categories have pretty much the same median (visually checked) - skin care, bath & body, beauty

In [ ]:
# check on df to vizz

cat_pirc = filtered20_df.groupby("category").agg({"price":["mean", "median", "std", "count"]}).sort_values(by='category')
cat_pirc.columns = ["_".join(tup) for tup in cat_pirc.columns]
cat_pirc.reset_index()

## Part 3: Investigating the Interplay Between Product Prices and Ratings

Objective: Analyze how product ratings (stars) correlate with product prices.

Correlation Coefficients:

Calculate the correlation coefficient between price and stars.
Is there a significant correlation between product price and its rating?

Visualizations:

Use a scatter plot to visualize the relationship between product rating and price. What patterns can you observe?
Use a correlation heatmap to visualize correlations between all numerical variables.
Examine if product prices typically follow a normal distribution using a QQ plot.


In [ ]:
pearson_corr = df_numerical["price"].corr(df_amz_ppd["stars"], method = "pearson")
print(pearson_corr)

In [ ]:
spearman_corr = df_numerical["price"].corr(df_amz_ppd["stars"], method = "spearman")
print(spearman_corr)

In [ ]:
sns.scatterplot(data = df_numerical, x = "price", y="stars")

plt.title("correlation between price an rating", fontsize=14)
plt.xticks(rotation=90)
plt.show()

In [ ]:
corr_matrix = df_numerical.corr()

plt.figure(figsize = (7,5))

sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm")
plt.show()

In [ ]:
import statsmodels.api as sm

sm.qqplot(cleaned_df["price"], line = "s")
plt.show()